In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.meshing import Mesh as NGMesh
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# Import mesh and gridfunction from 4_MagVekPot_HomCoil.ipynb

# Load mesh
ngmesh = NGMesh()
ngmesh.Load("results/homo/mesh_homo.vol")
mesh = Mesh(ngmesh)

# Load Gridfunction and B
order = 2
V = HCurl(mesh, order=order-1, nograds=True, dirichlet="outer")
gfA = GridFunction(V)
gfA.Load("results/homo/gfA_homo.vec")
B = curl(gfA)

In [3]:
# Konstanten
q_e = -1.602176634e-19  # Elektronenladung [C]
m_e = 9.10938356e-31    # Elektronenmasse [kg]

# Parameter außerhalb der Funktionen
dt = 1e-16               # Zeitschritt [s]
dt_safety = 1e-16
tolerance_z = 1e-4       # Toleranz um z=0
v0_min = 116700000              # Minimale Geschwindigkeit für Suche [m/s]
v0_max = 117018436           # Maximale Geschwindigkeit für Suche [m/s]

v_tolerance = 1       # Toleranz für Geschwindigkeitssuche [m/s]
progress_step = 10000     # Fortschrittsanzeige alle x Schritte

# Magnetfeldinterpolation
def magnetic_field(position):
    x, y, z = position
    point = mesh(x, y, z)
    B_at_point = B(point) 
    return np.array([B_at_point[0], B_at_point[1], B_at_point[2]])

# Bewegungsgleichung
def rhs(t, state):
    pos = state[:3]
    vel = state[3:]
    B_vec = magnetic_field(pos)  # Magnetfeld an aktueller Position
    acc = (q_e / m_e) * np.cross(vel, B_vec)  # Lorentzkraft: a = (q/m) * (v x B)
    return np.hstack((vel, acc))

# Runge-Kutta 4. Ordnung
def runge_kutta_4(state, dt, t):
    k1 = dt * rhs(t, state)
    k2 = dt * rhs(t + dt / 2, state + k1 / 2)
    k3 = dt * rhs(t + dt / 2, state + k2 / 2)
    k4 = dt * rhs(t + dt, state + k3)
    return state + (k1 + 2 * k2 + 2 * k3 + k4) / 6

# Dynamische Anpassung des Zeitschritts
def dynamic_dt(state, tolerance_z, dt_min=1e-19, dt_max=1e-16, factor=0.9):
    """
    Adjusts the time step size dynamically based on the distance of the state from a reference point.

    Parameters:
    state (list or array-like): The current state of the system, where state[2] represents the z-coordinate.
    tolerance_z (float): The tolerance value for the z-coordinate. If the distance is less than this value, the time step is adjusted.
    dt_min (float, optional): The minimum allowable time step size. Default is 1e-18.
    dt_max (float, optional): The maximum allowable time step size. Default is 1e-16.
    factor (float, optional): A scaling factor for adjusting the time step size. Default is 0.9.

    Returns:
    float: The adjusted time step size.
    """
    distance = abs(state[2])  # Entfernung von z = 0
    if distance < tolerance_z*100:
        return max(dt_min, factor * dt_max * (distance / tolerance_z))
    else:
        return dt_max
    
# Simulation der Elektronenbahn
def simulate_trajectory(s0, v0_z):
    """
    Simulates the trajectory of an electron starting at a given initial position and velocity.
    Parameters:
    s0 (array-like): Initial position of the electron as a 3-element array [x, y, z].
    v0_z (float): Initial velocity of the electron in the z-direction.
    Returns:
    tuple: A tuple containing:
        - trajectory (numpy.ndarray): Array of positions of the electron at each time step.
        - success (bool): True if the electron successfully reverses direction without crossing z=0, False otherwise.
        - reason (str): A string indicating the reason for the simulation ending. (overshoot, direction, success)
    The function uses a Runge-Kutta 4th order method to integrate the electron's motion over time. The simulation continues until one of the following conditions is met:
    - The electron reaches a position within a specified tolerance of z=0.
    - The electron crosses z=0.
    - The electron changes direction without reaching z=0.
    The function includes a safety check to ensure that the electron reverses direction correctly when it reaches z=0. If the electron successfully reverses direction, the function returns the trajectory and a success flag set to True. Otherwise, it returns the trajectory and a success flag set to False.
    """
    state = np.hstack((s0, [0, 0, v0_z]))  # Initialposition und -geschwindigkeit Elektron startet bei negative z mit positive z-Geschwindigkeit
    initial_state = np.copy(state)
    trajectory = [state[:3]]  # Liste der Positionen
    t = 0  # Startzeit
    safety_check_iterations = 100000 # Anzahl der Schritte, um sicherzustellen, dass das Elektron nicht weiterfliegt

    while True:
        dt = dynamic_dt(state, tolerance_z, dt_min=1e-19, dt_max=1e-13)
        
        state = runge_kutta_4(state, dt, t)  # RK4-Integration
        trajectory.append(state[:3])
        t += dt

        # TEST 
        #state = np.hstack(([0,0,-0.2], [0, 0, -34])) # Test: Elektron ändert geschwindigkeit ohne z=0 zu erreichen
        #state = np.hstack(([0,0,-0.2], [0, 0, 34])) # Test: Elektron fliegt über z=0 hinaus
        #state = np.hstack(([0,0,-0.000003], [0, 0, 0])) # Test: Elektron erreicht z=0 (Toleranz)
        
        # Feedback: Drucke Fortschritt
        if len(trajectory) % progress_step == 0:
            print(f"Zeit: {t:.2e}s | Position: {state[:3]} | Geschwindigkeit: {state[3:]}, t: {t}, dt: {dt}")

        # Abbruchbedingungen
        if abs(state[2]) <= tolerance_z:  # Elektron erreicht tolleranz um z=0
            print(f"!!! - Elektron erreicht z=0 (Toleranz) um. Letzte Position: {state[:3]}, Geschwindigkeit: {state[3:]} , t: {t}, dt: {dt}")
            print(f"Sicherheitsüberprüfung")
            
            safety_i = 0
            
            # Probiere  um sicherzustellen, dass das Elektron nicht weiterfliegt sondern umkehrt, heisst:
            # 1. position in z überschreitet nicht die Toleranz ->  status[2] < tolernace_z
            # 2. neue geschwidigkeit wird negativer                  ->  old_status[5] > new_status[5]
            while state[2] < tolerance_z:
                
                dt = dt_safety # Kleinerer Zeitschritt
                
                old_state = state # Speichere alten Zustand
                
                # Neue Position berechnen
                state = runge_kutta_4(state, dt, t)
                trajectory.append(state[:3])
                t += dt
                
                # Position und Geschwindigkeit Differenz
                #position_change = old_state[:3] - abs(state[:3]) 
                #velocity_change = old_state[3:] - abs(state[3:]) 
                
                # Feedback: Drucke Fortschritt
                # if safety_i % 100000 == 0: 
                #     #print("Safety check - ", safety_i, "Position change: ", position_change[2], "Geschwindigkeit change: ", velocity_change[2])
                #     print(f"Safety Check.    Position: {state[2]}, Geschwindigkeit: {state[5]} ")
                
                # Elektron fliegt weiter über z=0 hinaus -> Negativer Abbruch
                if state[2] > tolerance_z: 
                    print(f"Safety Check - Elektron fliegt über z=0 hinaus. Position: {state[2]}, Geschwindigkeit: {state[5]} ")
                    return np.array(trajectory), False, "overshoot"
                # Neue geschwidigkeit wird negativer
                elif state[5] < 0: 
                    safety_i += 1
                    print("enter")
                    continue
                # Elektron hat erfolgreich umgekehrt wenn er wieder eine Position in z überschreitet (bspw. initial_state[2] + 0.04 = -0.01)
                # position ist negativer
                # && 
                # geschwindigkeit ist negativ                   ->  Positiver Abbruch
                elif state[2] < (initial_state[2] + 0.04) and state[5] < 0: 
                    print(f"Safety Check - Elektron hat erfolgreich umgekehrt. Position: {state[:3]}, Geschwindigkeit: {state[3:]}")
                    return np.array(trajectory), True, "success"
                
        # Elektron fliegt weiter über z=0 hinaus -> Negativer Abbruch
        elif state[2] > tolerance_z: 
            print(f"Elektron fliegt über z=0 hinaus. Position: {state[:3]}, Geschwindigkeit: {state[3:]}, t: {t}, dt: {dt}")
            return np.array(trajectory), False, "overshoot"
        
        # Elektron ändert Richtung bzw. Geschwindigkeit in z, ohne z=0 zu erreichen -> Negativer Abbruch
        elif state[5] < 0: 
            print(f"Elektron ändert Richtung, ohne z=0 zu erreichen. Position: {state[:3]}, Geschwindigkeit: {state[3:]}, t: {t}, dt: {dt}")
            return np.array(trajectory), False, "direction"

# Suche nach der Anfangsgeschwindigkeit
def find_v0_z(s0):
    """
    Finds the initial velocity v0_z that results in an electron reversing direction at z=0.

    This function uses a binary search algorithm to find the initial velocity v0_z within a specified tolerance.
    It simulates the trajectory of an electron and adjusts the velocity bounds based on whether the electron
    reverses direction at z=0 or not.

    Parameters:
    s0 (float): The initial position of the electron.

    Returns:
    tuple: A tuple containing the found velocity v0_z (float), the trajectory (list), and a boolean indicating
           whether the velocity was found within the tolerance (True) or not (False).
    """
    global v0_min, v0_max
    iteration = 0

    
    while v0_max - v0_min > v_tolerance:
        iteration += 1
        v0_z = (v0_min + v0_max) / 2 
        print(f"Iteration {iteration}: Teste Geschwindigkeit v0_z = {v0_z:.3f} m/s", "  Position: ", s0)
        trajectory, success = simulate_trajectory(s0, v0_z)

        if success:  # Elektron kehrt bei z=0 um
            print(f"Geschwindigkeit {v0_z:.3f} m/s könnte passen (Elektron kehrt um).")
            v0_max = v0_z  # Erhöhe Oberegrenze
            break
        else:
            print(f"Geschwindigkeit {v0_z:.3f} m/s passt nicht (Richtung geändert oder fliegt über z=0 hinaus).")
            v0_min = v0_z  # Erhöhe Untergrenze
    if not (v0_max - v0_min > v_tolerance):
        print(f"---- Abbruch: Toleranz Geschwindigkeit: {v_tolerance} erreicht ----")
        return v0_z, trajectory, True
    else:
        print(f"Gefundene Geschwindigkeit: {v0_z:.3f} m/s")
        return v0_z, trajectory, False 

# Grobrastersuche, um bessere Startwerte zu finden
def coarse_search(s0, v0_min, v0_max, step):
    print("Starte Grobrastersuche...")
    for v in np.arange(v0_min, v0_max, step):
        print(f"Teste Geschwindigkeit: {v:.3f} m/s")
        _, success = simulate_trajectory(s0, v)
        if success:
            print(f"Grobrastersuche erfolgreich: Startwert gefunden bei v = {v:.3f} m/s")
            return v
    print("Grobrastersuche hat keine Lösung gefunden.")
    return None

# Suche nach der Anfangsgeschwindigkeit
def find_v0_z_with_raster(s0):
    global v0_min, v0_max
    step = (v0_max - v0_min) / 1e2  # Schrittweite für Raster-Suche
    print(f"Starte Geschwindigkeitssuche mit Raster-Suche (Schrittweite: {step:.3f} m/s, v0_min: {v0_min:.3f} m/s, v0_max: {v0_max:.3f} m/s)")
    v_start = coarse_search(s0, v0_min, v0_max, step)

    if v_start is not None:
        v0_min = v_start / 2
        v0_max = v_start * 2

    return find_v0_z(s0)

def find_v0_z_adaptive(s0):
    """
    Finds an initial velocity v0_z that results in an electron reversing direction at z=0,
    by adaptively modifying the velocity based on overshoot or premature direction change.

    Returns (v0_z, trajectory, success).
    """
    global v0_min, v0_max  # If you still want to keep track of some global bounds
    iteration = 0
    max_iterations = 50
    
    # Start with the midpoint or any initial guess
    v0_z = 0.5*(v0_min + v0_max)
    # Step size: for instance, 1/100 of the initial range
    step = 0.1 * (v0_max - v0_min)
    
    best_trajectory = None
    found_success = False
    
    while iteration < max_iterations:
        iteration += 1
        print(f"Iteration {iteration}: Teste Geschwindigkeit v0_z = {v0_z:.3f} m/s")
        
        trajectory, success, reason = simulate_trajectory(s0, v0_z)
        
        if success and reason == "success":
            print(f">>> Erfolg! Passende Geschwindigkeit gefunden: {v0_z:.3f} m/s")
            best_trajectory = trajectory
            found_success = True
            break
        else:
            # If no success, see why we failed
            if reason == "overshoot":
                # Velocity too high -> reduce velocity
                v0_z -= step
                print("    -> Overshoot: reducing velocity.")
            elif reason == "direction":
                # Velocity too low -> increase velocity
                v0_z += step
                print("    -> Changed direction prematurely: increasing velocity.")
            else:
                # Should not happen, but just in case
                print("    -> Unexpected reason, do something else...")

            # Optionally shrink step after each iteration to hone in
            step *= 0.5

            # Check if step is smaller than your desired velocity tolerance
            # if step < v_tolerance:
            #     print(f"Step size < velocity tolerance ({v_tolerance}). Stopping search.")
            #     break

            # Also keep v0_z in some bounds to avoid going too negative or too large
            if v0_z < 0:
                v0_z = 0.5 * (v0_min + v0_max)
                print("    -> v0_z negative, resetting to midpoint!")
            if v0_z > 1.1 * v0_max:
                v0_z = 0.5 * (v0_min + v0_max)
                print("    -> v0_z too large, resetting to midpoint!")
    
    # After the loop
    return v0_z, best_trajectory, found_success

def find_v0_z_bracket(s0):
    """
    Finds an initial velocity v0_z that results in the electron reversing around z=0,
    by bracketing the velocity between v0_min and v0_max.

    Adjust the global variables:
        v0_min, v0_max (starting bracket)
        v_tolerance (stop if bracket is smaller than this)
    as needed.

    Returns (v0_z, best_trajectory, found_success).
    """
    global v0_min, v0_max, v_tolerance
    max_iterations = 100  # or whatever you like
    found_success = False
    best_trajectory = None
    
    for iteration in range(1, max_iterations+1):
        # Midpoint of bracket
        v0_z = 0.5 * (v0_min + v0_max)

        print(f"Iteration {iteration}: testing v0_z = {v0_z:.6f} m/s "
              f"(v_min={v0_min:.6f}, v_max={v0_max:.6f})")

        trajectory, success, reason = simulate_trajectory(s0, v0_z)

        if success and reason == "success":
            print(f">>> SUCCESS at iteration {iteration} - velocity = {v0_z:.6f} m/s")
            found_success = True
            best_trajectory = trajectory
            break
        
        elif reason == "overshoot":
            # We overshot z=0 => velocity is too high => reduce upper bound
            v0_max = v0_z
            print(f"    -> Overshoot: new v_max = {v0_max:.6f}")
        
        elif reason == "direction":
            # Changed direction too soon => velocity is too low => raise lower bound
            v0_min = v0_z
            print(f"    -> Premature direction: new v_min = {v0_min:.6f}")
        
        else:
            # Should not normally happen, but just in case
            print("    -> Unexpected reason. Check your simulate_trajectory logic.")
        
        # Check bracket size
        if (v0_max - v0_min) < v_tolerance:
            print("Bracket is below velocity tolerance.")
            break
    
    return v0_z, best_trajectory, found_success
# Startposition
s0 = np.array([0, 0.007, -0.05])  # Elektron startet bei z = -0.01

# Bestimme minimale Geschwindigkeit für Umkehr bei z = 0
#v0_z, trajectory, success = find_v0_z_with_raster(s0)
#v0_z, trajectory, success = find_v0_z(s0)
#v0_z, trajectory, success = find_v0_z_adaptive(s0)
v0_z, trajectory, success = find_v0_z_bracket(s0)

if success:
    print(f"Benötigte Anfangsgeschwindigkeit: {v0_z:.3f} m/s")
else:
    print("Geschwindigkeit konnte nicht gefunden werden.")

Iteration 1: Teste Geschwindigkeit v0_z = 116859218.000 m/s
Elektron ändert Richtung, ohne z=0 zu erreichen. Position: [ 0.00030646  0.00461621 -0.00138515], Geschwindigkeit: [ 1.12232994e+08  3.26224553e+07 -1.02172925e+04], t: 6.303826008285039e-10, dt: 1.2468753206984602e-12
    -> Changed direction prematurely: increasing velocity.
Iteration 2: Teste Geschwindigkeit v0_z = 116891061.600 m/s
Elektron ändert Richtung, ohne z=0 zu erreichen. Position: [ 0.0006132   0.00468296 -0.00129395], Geschwindigkeit: [ 1.15553484e+08  1.77681770e+07 -2.03853208e+05], t: 6.327072312114498e-10, dt: 1.1645730921863248e-12
    -> Changed direction prematurely: increasing velocity.
Iteration 3: Teste Geschwindigkeit v0_z = 116906983.400 m/s
Elektron ändert Richtung, ohne z=0 zu erreichen. Position: [ 0.00033658  0.00462305 -0.00134475], Geschwindigkeit: [ 1.12749596e+08  3.09779306e+07 -8.68383065e+04], t: 6.304310402445402e-10, dt: 1.210418129523002e-12
    -> Changed direction prematurely: increasi

In [4]:
import plotly.graph_objects as go

# 3D-Plot erstellen
fig = go.Figure()


# Bahn des Elektrons hinzufügen
fig.add_trace(go.Scatter3d(
    x=trajectory[:, 0],
    y=trajectory[:, 1],
    z=trajectory[:, 2],
    mode='lines',
    name='Elektronenbahn',
    line=dict(color='blue', width=4)
))

# Startpunkt hinzufügen
fig.add_trace(go.Scatter3d(
    x=[s0[0]],
    y=[s0[1]],
    z=[s0[2]],
    mode='markers',
    name='Startposition',
    marker=dict(color='red', size=8)
))

# Layout anpassen
fig.update_layout(
    title="3D-Bahn des Elektrons im Magnetfeld",
    scene=dict(
        xaxis_title='x [m]',
        yaxis_title='y [m]',
        zaxis_title='z [m]'
    ),
    legend=dict(x=0.1, y=0.9)
)

# Plot anzeigen
fig.show()

TypeError: 'NoneType' object is not subscriptable